In [ ]:
%matplotlib notebook
import os
import time
import platform
import cv2

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import schemes_dev as sc
import pandas as pd

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": "Helvetica",
})

In [ ]:
class model_param:
    def __init__(self):
        self.adjoint = False    # Method of integration
        self.batch_time=30     # Time duration of training batch in terms of Δt steps
        self.batch_size=30     # Number of batches of training data
        self.lr = 1e-4         # Learning rate for model parameters
        self.lr_τ = 1e-5       # Leaning rate delay vector τ
        self.Nc = 40         # Maximum delay in terms of Δt steps
        self.niters=2000       # Maximum number of iterations
        self.test_freq=1       # Testing frequency in terms of iterations (for print and plot)
        self.viz=True          # Visualization
        self.savefig= True     # Set True to save figure
        self.folder='output_folder'   # folder name to store output
        self.use_gpu = False
        self.restart = False
        self.gpu = 0
        self.datafile = "path_to_data_file.txt"
        self.dimensions = 4
        
    def to(self, device):
        args_dir = dir(self)
        for var in args_dir:
            if type(getattr(self, var)) == int or type(getattr(self, var)) == int:
                exec('self.'+var+'=torch.tensor(self.'+var+').to(device)')
                #print(var+'='+ str(getattr(self,var)))


In [ ]:
base_fld = 'Lorenz'
param_df = pd.read_csv(base_fld+"/Param.csv")
column_name = list(param_df.keys())

In [ ]:
def fun_run(i):
    args = model_param()
    for var in column_name[1:]:
        #print('args.lr_τ=param_df.iloc['+str(i)+']["'+var+'"]')
        if var=='lr_tau':
            exec('args.lr_τ=param_df.iloc['+str(i)+']["'+var+'"]')
        else:
            exec('args.'+var+'=param_df.iloc['+str(i)+']["'+var+'"]')
    return args

In [ ]:
args_all = []
loss_all = []
delay_all = []

In [ ]:
for i in range(len(param_df)):
    args_all.append(fun_run(i))
    folder = args_all[i].folder
    loss_all.append(np.loadtxt(folder+'/loss.txt'))
    delay_all.append(np.loadtxt(folder+'/delay.txt'))

In [ ]:
diff = 'datafile' # How to catagorize data

# Loss plots

In [ ]:
catagory = {'Clean':[],'Noisy':[]}
catagory_keys = list(catagory.keys())

In [ ]:
for i in range(len(param_df)):
    loss_avg = np.mean(loss_all[i][-100:])
    dim = args_all[i].__dict__['dimensions']
    
    for key in catagory_keys: 
        if key.lower() in args_all[i].__dict__[diff].lower():
            catagory[key].append([dim, loss_avg])

In [ ]:
fig = plt.figure()
axs = fig.add_subplot(1,1,1)

color = ['b','r']

count = 0
for key in catagory_keys:
    
    arr_plot = np.array(catagory[key])
    axs.plot(arr_plot[:,0],arr_plot[:,1],'-'+color[count], label="$"+key+"$")
    axs.legend(fontsize=20)
    idx = np.where(arr_plot[:,1]==min(arr_plot[:,1]))[0][0]
    axs.plot(arr_plot[idx,0], arr_plot[idx,1],'*'+color[count], markersize=20)
    count += 1
    axs.set_xlabel('$Dimension$', fontsize=20)
    axs.set_ylabel('$\mathcal{L}$', fontsize=20)
    axs.set_xlim([min(arr_plot[:,0]),max(arr_plot[:,0])])
    
    fig.savefig(base_fld+'/Loss_compare.pdf')

In [ ]:
for i in range(len(param_df)):
    folder = args_all[i].folder
    fig = plt.figure()
    axs = fig.add_subplot(1,1,1)
    axs.plot(delay_all[i], linewidth = 3)
    
    axs.set_xlim([0,len(delay_all[i])])
    axs.set_xlabel('$Iterations$', fontsize=20)
    axs.set_ylabel('$\\tau$', fontsize=30)
    axs.tick_params(axis='both', which='major', labelsize=15)
    axs.tick_params(axis='both', which='minor', labelsize=8)
    plt.tight_layout()
    
    fig.savefig(folder+'/delay_plot.pdf')

# Make video

In [ ]:
import os
import moviepy.video.io.ImageSequenceClip as movie_maker
fps=5

In [ ]:
for i in range(len(param_df)):
    folder = args_all[i].folder
    print(folder)
    print(folder+'/video.avi')
    img_arr = []
    for j in range(0,args_all[i].niters,args_all[i].test_freq):
        img_path = folder+'/'+str(j)+'.png'
        if os.path.exists(img_path):
            img_arr.append(img_path)
    step = 1
    clip = movie_maker.ImageSequenceClip(img_arr[0::step], fps=fps)
    clip.write_videofile('./'+folder+'/video.mp4')

In [ ]:
def delayTS(x,tau,dim):
    xd = []
    s  = 0
    for i in range(dim):
        s  = i*tau 
        e  = len(x) - (dim-1-i)*tau
        #print('s=',s)
        #print('e=',e)
        if i == 0:
            xd = x[s:e].reshape(-1,1)
        else:
            xd = np.hstack((xd,x[s:e].reshape(-1,1)))
    return xd

def ami_fun(xg, tau_max):
    dim = 2
    ami = np.zeros(tau_max)
    eps  = np.finfo(float).eps
    
    x    = (xg-min(xg))
    x    = x*(1-eps)/max(x)
    
    n_bins = np.array(np.ceil(np.log2(len(x))), dtype=int)#//5
    #print(n_bins)
    
    x    = np.array(np.floor(x*n_bins), dtype=int)
    
    for tau in range(tau_max):
        pxy      = np.zeros((n_bins,n_bins))
        #print(pxy)
        
        xd       = delayTS(x,tau,dim)
        #print(xd)
        
        for xt in xd:
            pxy[xt[0], xt[1]] +=1 
        
        pxy = pxy/xd.shape[0] + eps
        
        px  = np.sum(pxy, axis = 1)
        py  = np.sum(pxy, axis = 0)
        
        pd  = np.outer(px,py)
        temp     = pxy/pd
        temp     = pxy*np.log2(temp)
        ami[tau] = np.sum(temp.reshape(-1))
    return ami

In [ ]:
def fnn_fun(x,dmax, tau, Rtol=10, Atol=2):
    eps = np.finfo(float).eps
    fnn = np.zeros(dmax)
    dim = []
    
    sigma = np.std(x)
    
    xzc = x-np.mean(x)              # zero centered x
    
    xc  = delayTS(xzc,tau,dim=1)    # current x delayed 
    
    for d in range(1,dmax+1):
        print('d=',d)
        dim.append(d)
        
        xn = delayTS(xzc,tau,d+1)
        xc = xc[:xn.shape[0],:]
        
        for j in range(xc.shape[0]):
            dist = np.sum((xc - xc[j])**2, axis=1)
            
            id_so = np.argsort(dist)
            
            for id_se in range(1,5):
                try:
                    id_ne = id_so[id_se]
                    dc_ne = dist[id_ne]       # current distance of nearest neighbor

                    dn_ne = dc_ne+ (xn[id_ne,-1]-xn[j,-1])**2 

                    Rc    = np.sqrt(dc_ne) + eps
                    Rn    = np.sqrt(dn_ne) + eps

                    if np.sqrt(dn_ne-dc_ne)/Rc > Rtol:
                        fnn[d-1] += 1
                    elif (Rn/sigma)>Atol:
                        fnn[d-1] += 1
                    break
                except:
                    print('exception occured for j=',j)
                        
        xc = xn    
    return dim, fnn
    

In [ ]:
for i in range(0,len(param_df),5):
    datafile = args_all[i].datafile
    folder = args_all[i].folder
    print(datafile)
    data = np.loadtxt(datafile)
    t = data[:,0]
    x = data[:,1]


    ami = ami_fun(x,500)

    fig = plt.figure()
    ax  = fig.add_subplot(1,1,1)

    ax.plot(t[:len(ami)],ami, c='red')
    ax.set_xlabel('$\\tau~(s)$'  , fontsize=20)
    ax.set_ylabel('$AMI~(bits)$' , fontsize=20)

    win =2
    for i in range(win,len(ami)-win):
        if (ami[i-1]-ami[i])>0 and ami[i+1]-ami[i]>0:
            id_min = i
            print(id_min)
            break

    tau_ami = id_min 
    ax.plot(t[tau_ami],ami[tau_ami],'v', label='$\\tau='+str("{:.2f}".format(t[tau_ami]))+'$')
    ax.legend(fontsize=20)
    
    fig.savefig(folder+'/AMI.pdf')
    
    dim, fnn = fnn_fun(x,10, tau_ami)
    
    fig = plt.figure()
    ax  = fig.add_subplot(1,1,1)

    ax.plot(dim,fnn,'-ob')

    fnn_zero = np.where(fnn==0)[0][0]
    print(fnn_zero)
    ax.plot(dim[fnn_zero],[0], 'vr', label='$Dimension='+str(dim[fnn_zero])+'$',markersize=10)
    ax.legend(fontsize=20)

    [ax.annotate('$'+str(int(fnn[i]))+'$',[dim[i],fnn[i]+500], fontsize=15) for i in range(len(fnn))]
    ax.set_xlabel('$Dimension$', fontsize=20)
    ax.set_ylabel('$FNN$', fontsize=20)
    #axs.set_xlim([min(arr_plot[:,0]),max(arr_plot[:,0])])

    fig.savefig(folder+'/FNN.pdf')

In [ ]:
datafile = args_all[5].datafile
print(datafile)
data = np.loadtxt(datafile)
fig = plt.figure()
axs = fig.add_subplot(projection='3d')

axs.plot(data[:,1], data[:,2],data[:,3])